# Target Allocation

In [1]:
import numpy as np
from scipy.optimize import minimize

volume = 1050.0
profit = 3000.0

number_of_product = 10

P = np.array([0.023, 0.031, 0.012, 0.022, 0.012, 0.034, 0.015, 0.028, 0.016, 0.024])
V_last_year = np.array([100.0, 200.0, 50.0, 50.0, 300.0, 20.0, 20.0, 60.0, 80.0, 120.0])
unit_product_profit = np.array([1.1, 2.4, 1.3, 3.8, 4.2, 2.5, 1.8, 1.5, 3.2, 2.2])

print("Last year sales amount is %d" % sum(V_last_year))
print("Last year profit is %d" % sum(V_last_year * unit_product_profit))

assert len(P) == number_of_product
assert len(unit_product_profit) == number_of_product
assert len(V_last_year) == number_of_product

Last year sales amount is 1000
Last year profit is 2801


In [2]:
# 目标函数
V = lambda U: V_last_year * (1 + P + U)
estimated_product_profit = lambda U: V_last_year * (1 + P + U) * unit_product_profit
profit_rate = lambda U: sum(V(U) * unit_product_profit) / sum(V(U))
reverse_profit_rate = lambda U: - profit_rate(U)

U = np.zeros(number_of_product)
print("Estimate next year's sales amount and profit")
print("Estimate profit rate is %f" % profit_rate(U))
print("Estimate sales amount is %f" % sum(V(U)))
print("Estimate sales amount for each product: ", V(U))
print("Estimate profit is %f" % sum(estimated_product_profit(U)))
print("Estimate profit for each product", estimated_product_profit(U))


Estimate next year's sales amount and profit
Estimate profit rate is 2.796028
Estimate sales amount is 1020.620000
Estimate sales amount for each product:  [102.3  206.2   50.6   51.1  303.6   20.68  20.3   61.68  81.28 122.88]
Estimate profit is 2853.682000
Estimate profit for each product [ 112.53   494.88    65.78   194.18  1275.12    51.7     36.54    92.52
  260.096  270.336]


## scipy.optimize.minimize

```python
>>> from scipy.optimize import minimize, rosen, rosen_der
>>> x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
>>> res = minimize(rosen, x0, method='Nelder-Mead', tol=1e-6)
>>> res.x
array([ 1.,  1.,  1.,  1.,  1.])
```

### constraints

Constraints definition (only for COBYLA and SLSQP). Each constraint is defined in a dictionary with fields:

type : str
Constraint type: ‘eq’ for equality, ‘ineq’ for inequality.

fun : callable
The function defining the constraint.

jac : callable, optional
The Jacobian of fun (only for SLSQP).

args : sequence, optional
Extra arguments to be passed to the function and Jacobian.


### bounds

Bounds for variables (only for L-BFGS-B, TNC and SLSQP). (min, max) pairs for each element in x, defining the bounds on that parameter. Use None for one of min or max when there is no bound in that direction.

### method

If not given, chosen to be one of BFGS, L-BFGS-B, SLSQP, depending if the problem has constraints or bounds.

In [3]:

total_volume_constraint = lambda U: sum(V(U)) - volume
profit_constraint = lambda U: sum(estimated_product_profit(U)) - profit

constraints = (
    {'type': 'ineq', 'fun': total_volume_constraint},
    {'type': 'ineq', 'fun': profit_constraint},
)
boundary = 0.1
bounds = (
    (-boundary, boundary), (-boundary, boundary), (-boundary, boundary), (-boundary, boundary), (-boundary, boundary),
    (-boundary, boundary), (-boundary, boundary), (-boundary, boundary), (-boundary, boundary), (-boundary, boundary)
)

res = minimize(reverse_profit_rate, U, method='SLSQP', tol=1e-6, bounds=bounds, constraints=constraints)

print("To meet our target (sales amount %d and profit %d), we need to tune the expect sales amount for each product" % (volume, profit))
U = res.x
print(U)
print("predicted increase rate are: ", P)
print("optimized increase rate are: ", U + P)
print("profit rate is %f" % profit_rate(U))
print("sales amount is %f" % sum(V(U)))
print("sales amount for each product: ", V(U))
print("predicted incremental sales: ", V(np.zeros(number_of_product)) - V_last_year)
print("incremental sales: ", V(U) - V_last_year)
print("profit is %f" % sum(estimated_product_profit(U)))
print("profit for each product", estimated_product_profit(U))

To meet our target (sales amount 1050 and profit 3000), we need to tune the expect sales amount for each product
[-0.1     0.0969 -0.1     0.1     0.1     0.1    -0.1    -0.1     0.1
 -0.1   ]
predicted increase rate are:  [0.023 0.031 0.012 0.022 0.012 0.034 0.015 0.028 0.016 0.024]
optimized increase rate are:  [-0.077   0.1279 -0.088   0.122   0.112   0.134  -0.085  -0.072   0.116
 -0.076 ]
profit rate is 2.875518
sales amount is 1050.000000
sales amount for each product:  [ 92.3  225.58  45.6   56.1  333.6   22.68  18.3   55.68  89.28 110.88]
predicted incremental sales:  [2.3  6.2  0.6  1.1  3.6  0.68 0.3  1.68 1.28 2.88]
incremental sales:  [-7.7  25.58 -4.4   6.1  33.6   2.68 -1.7  -4.32  9.28 -9.12]
profit is 3019.294000
profit for each product [ 101.53   541.392   59.28   213.18  1401.12    56.7     32.94    83.52
  285.696  243.936]
